# CNNs

This notebook contains the code needed to train and test the CNNs described in "Comparing deep learning and concept extraction based methods for patient phenotyping from clinical narratives".

## Preparation

First, the necessary libraries are imported. A connection to Google Drive is set up and the working directory is moved to "DLH project" (if someone shared this folder with you and not uploaded by you, you will need to make a shortcut to "DLH project" to use this code). It was specified that the GPU should be used for training, if one is available, a hyperparameter specifying the number of kernels per n-gram size was set (to 100, as that was the value used in the original paper), and torch was told to not use scientific notation for printing decimals. 

In [ ]:
import json
import matplotlib.pyplot as plt 
import numpy as np
import os
import pandas as pd
# import pycuda.driver as cuda
import time
import sys
import torch
import torch.nn as nn
import tqdm

from prettytable import PrettyTable
from sklearn.metrics import *
from torch.utils.data import Dataset

In [ ]:
# This is for figuring out some version information

# !pip install pycuda
# # https://medium.com/ai%C2%B3-theory-practice-business/use-gpu-in-your-pytorch-code-676a67faed09
# print('__Python VERSION:', sys.version)
# print('__pyTorch VERSION:', torch.__version__)
# print('__CUDA VERSION', )
# from subprocess import call
# # call(["nvcc", "--version"]) does not work
# ! nvcc --version
# print('__CUDNN VERSION:', torch.backends.cudnn.version())
# print('__Number CUDA Devices:', torch.cuda.device_count())
# print('__Devices')
# # call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
# print('Active CUDA Device: GPU', torch.cuda.current_device())
# print ('Available devices ', torch.cuda.device_count())
# print ('Current cuda device ', torch.cuda.current_device())

In [ ]:
# This is for figuring out some version information

# cuda.init()
# ## Get Id of default device
# torch.cuda.current_device() # Tesla T4 with standard GPU on Google drive
# # 0
# cuda.Device(0).name() # '0' is the id of your GPU

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/MyDrive/DLH project")

In [ ]:
kernels_per_n_gram_size = 100

In [ ]:
# For better printing
torch.set_printoptions(sci_mode=False)

## Loading the data
Here, we load the data to be used (produced in preprocessing_for_cnn.ipynb). labelled_corpus_df contains the labels and the cleaned text for each of the discharge summaries we will use, and study_corpus_tensor contains the representations of those documents, with study_corpus_tensor[i, :, :] representing the i-th document and study_corpus_tensor[i, j, :] containing the word embedding vector for the j-th word in the i-th document.

The data is put into a custom dataset object for use in PyTorch.

In [ ]:
labelled_corpus_df = pd.read_csv("labelled_corpus_df.csv", index_col = 0)

In [ ]:
labelled_corpus_df.head(1)

,HADM_ID,SUBJECT_ID,Advanced.Cancer,Advanced.Heart.Disease,Advanced.Lung.Disease,Chronic.Neurological.Dystrophies,Chronic.Pain.Fibromyalgia,Alcohol.Abuse,Other.Substance.Abuse,Obesity,Schizophrenia.and.other.Psychiatric.Disorders,Depression,Cleaned Text
0,118003.0,3644,0,0,0,0,1,0,0,0,0,1,admission date 2200 4 7 discharge date 2200 4 ...


In [ ]:
# Find the number of occurrences of each condition
labelled_corpus_df[["Advanced.Cancer",
                  "Advanced.Heart.Disease",
                  "Advanced.Lung.Disease",
                  "Chronic.Neurological.Dystrophies",
                  "Chronic.Pain.Fibromyalgia",
                  "Alcohol.Abuse",
                  "Other.Substance.Abuse",
                  "Obesity",
                  "Schizophrenia.and.other.Psychiatric.Disorders",
                  "Depression"]].sum(axis = 0)

Advanced.Cancer                                  147
Advanced.Heart.Disease                           221
Advanced.Lung.Disease                            141
Chronic.Neurological.Dystrophies                 308
Chronic.Pain.Fibromyalgia                        277
Alcohol.Abuse                                    172
Other.Substance.Abuse                            139
Obesity                                           99
Schizophrenia.and.other.Psychiatric.Disorders    248
Depression                                       388
dtype: int64

In [ ]:
# Takes some time (< 1 minute)
study_corpus_tensor = torch.load("embedded_docs.pt")

In [ ]:
study_corpus_tensor.shape

torch.Size([1341, 5434, 100])

In [ ]:
study_corpus_tensor.device

device(type='cpu')

In [ ]:
study_corpus_tensor = study_corpus_tensor.to(device)
print(study_corpus_tensor.device)

cuda:0


In [ ]:
embedding_vector_size = study_corpus_tensor.shape[2]
print(embedding_vector_size)

100


In [ ]:
class CustomDatasetEmbedded(Dataset):
    def __init__(self, corpus_tensor, labels):
        """
        Store the corpus (of shape num_docs by max_num_words_per_doc by size_of_word embedding) 
        labels (for a single target variable)
        """

        self.x = corpus_tensor
        self.y = labels

    def __len__(self):

        """
        Return the number of documents
        """
        return len(self.y)

    def __getitem__(self, index):
        """
        Return one document (represented as a tensor, with each row being the embedding for one word in that document),
        and its label (whether the patient described has or does not have some phenotype)
        """
        return (self.x[index, :, :], self.y[index])

In [ ]:
depression_y = torch.tensor(labelled_corpus_df["Depression"]).to(device)
print(depression_y.device)

cuda:0


In [ ]:
depression_dataset = CustomDatasetEmbedded(study_corpus_tensor, depression_y)

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(depression_dataset, [0.8, 0.2])

In [ ]:
len(train_dataset)

In [ ]:
len(test_dataset)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True) # Batch size?
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32)

# CNN
This class allows you to create CNNs for predicting patient characterstics based on discharge summaries as described in the paper being replicated. n_gram_sizes must be a list which can contain the integers 1, 2, 3, 4, and/or 5. The CNN created will consider n-grams of the sizes specified in n_gram_sizes when looking at discharge summaries. Each CNN is a binary classifier which predicts if a patient has or lacks a single condition.

In [ ]:
class CNN_n_gram(nn.Module):
    def __init__(self, n_gram_sizes):
        super(CNN_n_gram, self).__init__()

        self.n_gram_sizes = n_gram_sizes
        num_sizes = 0

        if 1 in self.n_gram_sizes:
            self.conv1 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (1, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv1.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv1.bias)

            # each kernel's feature map is condensed to a single value
            conv1_output_height = study_corpus_tensor.shape[1] + 1 - 1
            self.pool1 = nn.MaxPool2d(kernel_size = (conv1_output_height, 1))
            num_sizes += 1

        if 2 in self.n_gram_sizes:
            self.conv2 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (2, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv2.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv2.bias)

            conv2_output_height = study_corpus_tensor.shape[1] + 1 - 2
            self.pool2 = nn.MaxPool2d(kernel_size = (conv2_output_height, 1))
            num_sizes += 1

        if 3 in self.n_gram_sizes:
            self.conv3 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (3, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv3.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv3.bias)

            conv3_output_height = study_corpus_tensor.shape[1] + 1 - 3
            self.pool3 = nn.MaxPool2d(kernel_size = (conv3_output_height, 1))
            num_sizes += 1

        if 4 in self.n_gram_sizes:
            self.conv4 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (4, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv4.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv4.bias)

            conv4_output_height = study_corpus_tensor.shape[1] + 1 - 4
            self.pool4 = nn.MaxPool2d(kernel_size = (conv4_output_height, 1))
            num_sizes += 1

        if 5 in self.n_gram_sizes:
            self.conv5 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (5, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv5.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv5.bias)

            conv5_output_height = study_corpus_tensor.shape[1] + 1 - 5
            self.pool5 = nn.MaxPool2d(kernel_size = (conv5_output_height, 1))
            num_sizes += 1

        assert num_sizes > 0 # I can't just assert that len(n_gram_sizes) > 0, as n_gram_sizes may contain unsupported values

        self.do = nn.Dropout(p = 0.5) 

        self.fc = nn.Linear(kernels_per_n_gram_size * num_sizes, 2)
        # torch.nn.init.normal_(self.fc.weight, mean = 0, std = 0.01) # This seemed to be done in the original code (written in Lua, hard to understand)
        # torch.nn.init.zeros_(self.fc.bias)                          # But it badly hurts performance here

        self.activation = nn.LogSoftmax(dim = 1) # Called on tensor of shape [batch_size, 2]
        
    def forward(self, x):
        # Add an extra dimension, because nn.Conv2d takes an input of (batch_size, number of channels, height of input, width of input)
        # and not (batch_size, height of input, width of input)
        x = torch.unsqueeze(x, dim = 1)

        list_of_convolutional_outputs = []

        # For each n-gram size, apply the filters to the input, apply ReLU, then perform global max
        # pooling to condense the output of each filter to a single value
        # Adding the outputs of the convolutional layer and the following ReLU and pooling operations to a list
        # enables this class to be used for CNNs considering many different combinations of n-gram sizes
        if 1 in self.n_gram_sizes:
            x1 = self.conv1(x)
            x1 = torch.relu(x1)
            x1 = self.pool1(x1)
            list_of_convolutional_outputs.append(x1)
        
        if 2 in self.n_gram_sizes:
            x2 = self.conv2(x)
            x2 = torch.relu(x2)
            x2 = self.pool2(x2)
            list_of_convolutional_outputs.append(x2)
        
        if 3 in self.n_gram_sizes:
            x3 = self.conv3(x)
            x3 = torch.relu(x3)
            x3 = self.pool3(x3)
            list_of_convolutional_outputs.append(x3)

        if 4 in self.n_gram_sizes:
            x4 = self.conv4(x)
            x4 = torch.relu(x4)
            x4 = self.pool4(x4)
            list_of_convolutional_outputs.append(x4)
        
        if 5 in self.n_gram_sizes:
            x5 = self.conv5(x)
            x5 = torch.relu(x5)
            x5 = self.pool5(x5)
            list_of_convolutional_outputs.append(x5)

        # Combine the results of applying differently sized kernels in parallel
        x = torch.cat(list_of_convolutional_outputs, dim = 1)

        # Perform a dropout to reduce overfitting
        x = self.do(x)

        # To deal with shape differences
        x = torch.flatten(x, start_dim = 1)
        
        # Feed the (dropped out) outputs of the pooling step into a fully connected layer,
        # and then a log-softmax activation function, to generate two values
        # If the first is greater, the patient is classified as not having the condition
        # otherwise, they are classified as having it. 
        x = self.fc(x)
        x = self.activation(x)

        return x

In [ ]:
# cnn_1_gram_model = CNN_n_gram([1]).to(device)
# print(cnn_1_gram_model)
cnn_1_2_gram_model = CNN_n_gram([1, 2]).to(device)
print(cnn_1_2_gram_model)
# cnn_1_2_3_gram_model = CNN_n_gram([1, 2, 3]).to(device)
# print(cnn_1_2_3_gram_model)

CNN_n_gram(
  (conv1): Conv2d(1, 100, kernel_size=(1, 100), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(5434, 1), stride=(5434, 1), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(1, 100, kernel_size=(2, 100), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=(5433, 1), stride=(5433, 1), padding=0, dilation=1, ceil_mode=False)
  (do): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=200, out_features=2, bias=True)
  (activation): LogSoftmax(dim=1)
)


In [ ]:
for name, param in cnn_1_2_gram_model.named_parameters():
    print(name)
    print(param.shape)

conv1.weight
torch.Size([100, 1, 1, 100])
conv1.bias
torch.Size([100])
conv2.weight
torch.Size([100, 1, 2, 100])
conv2.bias
torch.Size([100])
fc.weight
torch.Size([2, 200])
fc.bias
torch.Size([2])


In [ ]:
# This loss function was specified in main.lua within the GitHub repository provided by the authors of the original study
# local criterion = nn.ClassNLLCriterion()
criterion = nn.modules.loss.NLLLoss() # CrossEntropyLoss() doesn't help

# Adadelta was specified in a PDF file attached to the original paper
# The hyperparameters for the optimizer are specified in trainer.lua from the GitHub repository provided by the authors of the original study
optimizer = torch.optim.Adadelta(cnn_1_2_3_gram_model.parameters(), rho = 0.95, eps = 1e-6)

This function is responsible for training CNNs. It takes in a CNN, a training dataloader, the number of epochs of training to complete, an optmizer, and a loss function, and returns the trained model.

You can uncomment the line with tqdm in it to see the progress within each loop (it is currently replaced with a line without tqdm, as printing that when training 60 different CNNs would be excessively verbose).

In [ ]:
# From main.lua in the provided code:
# cmd:option('-epochs', 20, 'Number of training epochs')
n_epochs = 20

# From HW3 CNN
def train_model(model, train_dataloader, n_epoch, optimizer, criterion, normalize = True):
    """
    :param model: A CNN model
    :param train_dataloader: the DataLoader of the training data
    :param n_epoch: number of epochs to train
    :return:
        model: trained model
    """
    model.train() # prep model for training
    
    
    for epoch in range(n_epoch):
        curr_epoch_loss = []
        # For testing
        loader_index = 0
        # for x, y in tqdm.tqdm(train_dataloader):
        for x, y in train_dataloader:
            """
            TODO: Within the loop, do the normal training procedures:
                   pass the input through the model
                   pass the output through loss_func to compute the loss (name the variable as *loss*)
                   zero out currently accumulated gradient, use loss.basckward to backprop the gradients, then call optimizer.step
            """
            # FROM HOMEWORK 2
            """ Step 1. clear gradients """
            optimizer.zero_grad()
            """ 
            TODO: Step 2. perform forward pass using `model`, save the output to y_hat;
                  Step 3. calculate the loss using `criterion`, save the output to loss.
            """

            y_hat = model(x)
            loss = criterion(y_hat, y)

            """ Step 4. backward pass """
            loss.backward()
            """ Step 5. optimization """
            optimizer.step()

            if normalize:
                # Linear layer's weights (for unigram CNN) are of size [2, 100]
                # Normalize each of the two rows to a L2 norm of 3?
                norms = torch.linalg.vector_norm(model.fc.weight, dim = 1, keepdim = True) #keepdim = True)
                model.fc.weight = torch.nn.Parameter((3 * model.fc.weight) / (norms + 1e-7)) # LIKE SO????

            """ Step 6. record loss """
            curr_epoch_loss.append(loss.cpu().data.numpy())


            loader_index += 1 # For testing purposes

        # print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}") # Can include if you are training a single model
    return model

In [ ]:
# Try training a single model:

# start_time = time.time()
# cnn_1_2_gram_model = train_model(model = cnn_1_2_gram_model,
#                                  train_dataloader = train_loader,
#                                  n_epoch = n_epochs,
#                                  optimizer = optimizer,
#                                  criterion = criterion)
# end_time = time.time()
# print()
# print(end_time - start_time)

This function evaluates a trained model on the data present within a test dataloader. It returns the true labels, predicted labels, and output of the CNN corresponding to the 1 class (corresponding to the patient having the condition).

In [ ]:
# From HW3 CNN
def eval_model(model, dataloader):
    model.eval()
    Y_pred  = []
    Y_true  = []
    Y_score = []
    with torch.no_grad():
        for x, y in dataloader:
            # your code here
            Y_true.append(y)
            
            y_hat = model(x)
            
            Y_score.append(y_hat[:, 1])

            # https://campuswire.com/c/G902DEAF1/feed/823
            # Return class with higher probability
            Y_pred.append(torch.max(y_hat, 1).indices)
            
    Y_score = [y_score.to("cpu") for y_score in Y_score]
    Y_pred  = [y_pred.to("cpu")  for y_pred  in Y_pred]
    Y_true  = [y_true.to("cpu")  for y_true  in Y_true]

    Y_score = np.concatenate(Y_score, axis = 0)    
    Y_pred  = np.concatenate(Y_pred,  axis=0)
    Y_true  = np.concatenate(Y_true,  axis=0)

    return Y_score, Y_pred, Y_true

In [ ]:
# Test a single model
# y_score, y_pred, y_true = eval_model(cnn_1_2_gram_model, test_loader)

In [ ]:
# print("Predicted percent of patients that have the condition:", np.sum(y_pred) / len(y_pred))
# print("Actual percent of patients that have the condition:", np.sum(y_true) / len(y_true))
# print("Accuracy:", accuracy_score(y_true, y_pred))
# print("Precision:", precision_score(y_true, y_pred))
# print("Recall:", recall_score(y_true, y_pred))
# print("F1 Score:", f1_score(y_true, y_pred))
# print("AUC:", roc_auc_score(y_true, y_score))

In [ ]:
# See how the returned outputs are distributed
# plt.boxplot(y_score)
# plt.title("Seed = " + str(seed))
# plt.show()

In [ ]:
# From https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/23?page=2

# def count_parameters(model):
#     table = PrettyTable(["Modules", "Parameters"])
#     total_params = 0
#     for name, parameter in model.named_parameters():
#         if not parameter.requires_grad: 
#             continue
#         param = parameter.numel()
#         table.add_row([name, param])
#         total_params+=param
#     print(table)
#     print(f"Total Trainable Params: {total_params}")
#     return total_params

# count_parameters(cnn_1_2_gram_model)

This function trains a CNN which considers the n-gram sizes described in the list n_gram_sizes for detecting the provided condition. You must provide a training dataloader, test dataloader, the device to use for training, and the number of epochs of training to perform.

It will print information about training time and performance, and return that information in a dictionary.

You can use this function to test out a single model.

In [ ]:
def train_test_model(n_gram_sizes, condition, train_loader, test_loader, device, n_epochs):
    results_dict = {}

    which_model = str(n_gram_sizes) + "-gram CNN for " + condition
    print(which_model)

    model = CNN_n_gram(n_gram_sizes).to(device)

    criterion = nn.modules.loss.NLLLoss()

    optimizer = torch.optim.Adadelta(model.parameters(), rho = 0.95, eps = 1e-6)

    start_time = time.time()
    model = train_model(model = model,
                        train_dataloader = train_loader,
                        n_epoch = n_epochs,
                        optimizer = optimizer,
                        criterion = criterion)
    end_time = time.time()
    training_time = end_time - start_time
    training_time_statement = "Training time: " + str(training_time) + " seconds"
    print(training_time_statement)
    results_dict["training time"] = training_time

    y_score, y_pred, y_true = eval_model(model, test_loader)

    predicted_positive = np.sum(y_pred) / len(y_pred)
    predicted_positive_statement = "Predicted percent of patients that have the condition: " + str(predicted_positive)
    print(predicted_positive_statement)
    results_dict["predicted percent positive"] = predicted_positive

    actual_positive = np.sum(y_true) / len(y_true)
    actual_positive_statement = "Actual percent of patients that have the condition: " + str(actual_positive)
    print(actual_positive_statement)
    results_dict["actual percent positive"] = actual_positive

    accuracy = accuracy_score(y_true, y_pred)
    accuracy_statement = "Accuracy: " + str(accuracy)
    print(accuracy_statement)
    results_dict["accuracy"] = accuracy

    precision = precision_score(y_true, y_pred)
    precision_statement = "Precision: " + str(precision)
    print(precision_statement)
    results_dict["precision"] = precision

    recall = recall_score(y_true, y_pred)
    recall_statement = "Recall: " + str(recall)
    print(recall_statement)
    results_dict["recall"] = recall

    f1 = f1_score(y_true, y_pred)
    f1_statement = "F1 Score: " + str(f1)
    print(f1_statement)
    results_dict["f1"] = f1

    auc = roc_auc_score(y_true, y_score)
    auc_statement = "AUC: " + str(auc)
    print(auc_statement)
    results_dict["auc"] = auc

    print()
    return results_dict

Here, we figure out the name of the file to store results to.

In [ ]:
max_index_results_file = 0
for filename in os.listdir():
    split_filename = filename.split(".")
    if split_filename[0].startswith("results_") and split_filename[1] == "json":
        index = int(split_filename[0][8:])
        if index > max_index_results_file:
            max_index_results_file = index

new_file_name = "results_" + str(max_index_results_file + 1) + ".json"
print(new_file_name)

results_7.json


Here, we train and test 6 models considering different sets of n-grams for each of the 10 conditions. The information is stored into a JSON file. It takes a bit over 1 hour and 10 minutes total, on Google Colab with regular RAM and regular GPU acceleration.

In [ ]:
seed = 0
torch.manual_seed(seed)

all_conditions = ["Advanced.Cancer",
                  "Advanced.Heart.Disease",
                  "Advanced.Lung.Disease",
                  "Chronic.Neurological.Dystrophies",
                  "Chronic.Pain.Fibromyalgia",
                  "Alcohol.Abuse",
                  "Other.Substance.Abuse",
                  "Obesity",
                  "Schizophrenia.and.other.Psychiatric.Disorders",
                  "Depression"]

overall_results_dict = {}

for condition in all_conditions:
    overall_results_dict[condition] = {}

    labels = torch.tensor(labelled_corpus_df[condition]).to(device)

    dataset = CustomDatasetEmbedded(study_corpus_tensor, labels)

    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True) # Batch size?
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32)

    overall_results_dict[condition]["[1]"] = train_test_model(n_gram_sizes = [1],
                                                              condition = condition,
                                                              train_loader = train_loader,
                                                              test_loader = test_loader,
                                                              device = device,
                                                              n_epochs = n_epochs)
    overall_results_dict[condition]["[1, 2]"] = train_test_model(n_gram_sizes = [1, 2],
                                                                 condition = condition,
                                                                 train_loader = train_loader,
                                                                 test_loader = test_loader,
                                                                 device = device,
                                                                 n_epochs = n_epochs)
    overall_results_dict[condition]["[1, 2, 3]"] = train_test_model(n_gram_sizes = [1, 2, 3],
                                                                    condition = condition,
                                                                    train_loader = train_loader,
                                                                    test_loader = test_loader,
                                                                    device = device,
                                                                    n_epochs = n_epochs)
    overall_results_dict[condition]["[1, 2, 3, 4]"] = train_test_model(n_gram_sizes = [1, 2, 3, 4],
                                                                       condition = condition,
                                                                       train_loader = train_loader,
                                                                       test_loader = test_loader,
                                                                       device = device,
                                                                       n_epochs = n_epochs)
    overall_results_dict[condition]["[1, 2, 3, 4, 5]"] = train_test_model(n_gram_sizes = [1, 2, 3, 4, 5],
                                                                          condition = condition,
                                                                          train_loader = train_loader,
                                                                          test_loader = test_loader,
                                                                          device = device,
                                                                          n_epochs = n_epochs)
    overall_results_dict[condition]["[2, 3, 4, 5]"] = train_test_model(n_gram_sizes = [2, 3, 4, 5],
                                                                       condition = condition,
                                                                       train_loader = train_loader,
                                                                       test_loader = test_loader,
                                                                       device = device,
                                                                       n_epochs = n_epochs)

with open(new_file_name, 'w') as json_file:
    json.dump(overall_results_dict, json_file)

[1]-gram CNN for Advanced.Cancer
Training time: 9.548596620559692 seconds
Predicted percent of patients that have the condition: 0.05970149253731343
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.9216417910447762
Precision: 0.875
Recall: 0.42424242424242425
F1 Score: 0.5714285714285714
AUC: 0.9455834945196647

[1, 2]-gram CNN for Advanced.Cancer
Training time: 23.43425226211548 seconds
Predicted percent of patients that have the condition: 0.08955223880597014
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.9365671641791045
Precision: 0.8333333333333334
Recall: 0.6060606060606061
F1 Score: 0.7017543859649124
AUC: 0.9423597678916829

[1, 2, 3]-gram CNN for Advanced.Cancer
Training time: 58.291778326034546 seconds
Predicted percent of patients that have the condition: 0.08955223880597014
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.9291044776119403
Precision: 0.7916666666666666


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 22.927724838256836 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8768656716417911
Precision: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Recall: 0.0
F1 Score: 0.0
AUC: 0.8927143778207609

[1, 2, 3]-gram CNN for Advanced.Lung.Disease
Training time: 58.59787917137146 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8768656716417911
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.9092198581560285

[1, 2, 3, 4]-gram CNN for Advanced.Lung.Disease


...

Training time: 98.60812139511108 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.09328358208955224
Accuracy: 0.9067164179104478
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.6888888888888889

[1, 2, 3, 4, 5]-gram CNN for Obesity


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 129.01729226112366 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.09328358208955224
Accuracy: 0.9067164179104478
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.6788477366255145

[2, 3, 4, 5]-gram CNN for Obesity


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 120.9954400062561 seconds
Predicted percent of patients that have the condition: 0.011194029850746268
Actual percent of patients that have the condition: 0.09328358208955224
Accuracy: 0.9104477611940298
Precision: 0.6666666666666666
Recall: 0.08
F1 Score: 0.14285714285714285
AUC: 0.7506172839506172

[1]-gram CNN for Schizophrenia.and.other.Psychiatric.Disorders
Training time: 9.336776971817017 seconds
Predicted percent of patients that have the condition: 0.007462686567164179
Actual percent of patients that have the condition: 0.19029850746268656
Accuracy: 0.8171641791044776
Precision: 1.0
Recall: 0.0392156862745098
F1 Score: 0.07547169811320754
AUC: 0.787295563386645

[1, 2]-gram CNN for Schizophrenia.and.other.Psychiatric.Disorders
Training time: 22.93264937400818 seconds
Predicted percent of patients that have the condition: 0.0708955223880597
Actual percent of patients that have the condition: 0.19029850746268656
Accuracy: 0.8507462686567164
Precision: 0.789473684210

## Ablation

Here, we try removing the ReLU applied to the output of the convolutional layers. We thought that it was kind of pointless, as a global max pooling operation is applied to the output of each kernel, so turning all negative values into 0 in feature maps shouldn't do anything, as long as the feature map contains a single positive value.

In [ ]:
class CNN_n_gram_no_ReLU(nn.Module):
    def __init__(self, n_gram_sizes):
        super(CNN_n_gram_no_ReLU, self).__init__()

        self.n_gram_sizes = n_gram_sizes
        num_sizes = 0

        if 1 in self.n_gram_sizes:
            self.conv1 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (1, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv1.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv1.bias)

            # each kernel's feature map is condensed to a single value
            conv1_output_height = study_corpus_tensor.shape[1] + 1 - 1
            self.pool1 = nn.MaxPool2d(kernel_size = (conv1_output_height, 1))
            num_sizes += 1

        if 2 in self.n_gram_sizes:
            self.conv2 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (2, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv2.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv2.bias)

            conv2_output_height = study_corpus_tensor.shape[1] + 1 - 2
            self.pool2 = nn.MaxPool2d(kernel_size = (conv2_output_height, 1))
            num_sizes += 1

        if 3 in self.n_gram_sizes:
            self.conv3 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (3, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv3.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv3.bias)

            conv3_output_height = study_corpus_tensor.shape[1] + 1 - 3
            self.pool3 = nn.MaxPool2d(kernel_size = (conv3_output_height, 1))
            num_sizes += 1

        if 4 in self.n_gram_sizes:
            self.conv4 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (4, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv4.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv4.bias)

            conv4_output_height = study_corpus_tensor.shape[1] + 1 - 4
            self.pool4 = nn.MaxPool2d(kernel_size = (conv4_output_height, 1))
            num_sizes += 1

        if 5 in self.n_gram_sizes:
            self.conv5 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (5, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)
            torch.nn.init.uniform_(self.conv5.weight, -0.01, 0.01)
            torch.nn.init.zeros_(self.conv5.bias)

            conv5_output_height = study_corpus_tensor.shape[1] + 1 - 5
            self.pool5 = nn.MaxPool2d(kernel_size = (conv5_output_height, 1))
            num_sizes += 1

        assert num_sizes > 0 # I can't just assert that len(n_gram_sizes) > 0, as n_gram_sizes may contain unsupported values

        self.do = nn.Dropout(p = 0.5) 

        self.fc = nn.Linear(kernels_per_n_gram_size * num_sizes, 2)
        # torch.nn.init.normal_(self.fc.weight, mean = 0, std = 0.01)
        # torch.nn.init.zeros_(self.fc.bias)

        self.activation = nn.LogSoftmax(dim = 1) # Called on tensor of shape [batch_size, 2]
        
    def forward(self, x):
        # Add an extra dimension, because nn.Conv2d takes an input of (batch_size, number of channels, height of input, width of input)
        # and not (batch_size, height of input, width of input)
        x = torch.unsqueeze(x, dim = 1)

        list_of_convolutional_outputs = []

        # For each n-gram size, apply the filters to the input, then perform global max
        # pooling to condense the output of each filter to a single value
        # Adding the outputs of the convolutional layer and the following pooling operation to a list
        # enables this class to be used for CNNs considering many different combinations of n-gram sizes
        if 1 in self.n_gram_sizes:
            x1 = self.conv1(x)
            x1 = self.pool1(x1)
            list_of_convolutional_outputs.append(x1)
        
        if 2 in self.n_gram_sizes:
            x2 = self.conv2(x)
            x2 = self.pool2(x2)
            list_of_convolutional_outputs.append(x2)
        
        if 3 in self.n_gram_sizes:
            x3 = self.conv3(x)
            x3 = self.pool3(x3)
            list_of_convolutional_outputs.append(x3)

        if 4 in self.n_gram_sizes:
            x4 = self.conv4(x)
            x4 = self.pool4(x4)
            list_of_convolutional_outputs.append(x4)
        
        if 5 in self.n_gram_sizes:
            x5 = self.conv5(x)
            x5 = self.pool5(x5)
            list_of_convolutional_outputs.append(x5)

        # Combine the results of applying differently sized kernels in parallel
        x = torch.cat(list_of_convolutional_outputs, dim = 1)

        # Perform a dropout to reduce overfitting
        x = self.do(x)

        x = torch.flatten(x, start_dim = 1)

        # Feed the (dropped out) outputs of the pooling step into a fully connected layer,
        # and then a log-softmax activation function, to generate two values
        # If the first is greater, the patient is classified as not having the condition
        # otherwise, they are classified as having it. 
        x = self.fc(x)
        x = self.activation(x)

        return x

In [ ]:
# Analogous to train_test_model() earlier
def train_test_model_no_ReLU(n_gram_sizes, condition, train_loader, test_loader, device, n_epochs):
    results_dict = {}

    which_model = str(n_gram_sizes) + "-gram CNN for " + condition
    print(which_model)

    model = CNN_n_gram_no_ReLU(n_gram_sizes).to(device)

    criterion = nn.modules.loss.NLLLoss()

    optimizer = torch.optim.Adadelta(model.parameters(), rho = 0.95, eps = 1e-6)

    start_time = time.time()
    model = train_model(model = model,
                        train_dataloader = train_loader,
                        n_epoch = n_epochs,
                        optimizer = optimizer,
                        criterion = criterion)
    end_time = time.time()
    training_time = end_time - start_time
    training_time_statement = "Training time: " + str(training_time) + " seconds"
    print(training_time_statement)
    results_dict["training time"] = training_time

    y_score, y_pred, y_true = eval_model(model, test_loader)

    predicted_positive = np.sum(y_pred) / len(y_pred)
    predicted_positive_statement = "Predicted percent of patients that have the condition: " + str(predicted_positive)
    print(predicted_positive_statement)
    results_dict["predicted percent positive"] = predicted_positive

    actual_positive = np.sum(y_true) / len(y_true)
    actual_positive_statement = "Actual percent of patients that have the condition: " + str(actual_positive)
    print(actual_positive_statement)
    results_dict["actual percent positive"] = actual_positive

    accuracy = accuracy_score(y_true, y_pred)
    accuracy_statement = "Accuracy: " + str(accuracy)
    print(accuracy_statement)
    results_dict["accuracy"] = accuracy

    precision = precision_score(y_true, y_pred)
    precision_statement = "Precision: " + str(precision)
    print(precision_statement)
    results_dict["precision"] = precision

    recall = recall_score(y_true, y_pred)
    recall_statement = "Recall: " + str(recall)
    print(recall_statement)
    results_dict["recall"] = recall

    f1 = f1_score(y_true, y_pred)
    f1_statement = "F1 Score: " + str(f1)
    print(f1_statement)
    results_dict["f1"] = f1

    auc = roc_auc_score(y_true, y_score)
    auc_statement = "AUC: " + str(auc)
    print(auc_statement)
    results_dict["auc"] = auc

    print()
    return results_dict

In [ ]:
# Print the results of training and testing all 60 CNNs with ReLU omitted, and save them to "no_ReLU_results.json"

seed = 0
torch.manual_seed(seed)

overall_results_dict_no_relu = {}

# advanced or metastatic cancer, advanced heart disease, advanced lung disease, chronic neurologic dystrophies, chronic pain, alcohol abuse, substance abuse, obesity, psychiatric disorders, or depression.
all_conditions = ["Advanced.Cancer",
                  "Advanced.Heart.Disease",
                  "Advanced.Lung.Disease",
                  "Chronic.Neurological.Dystrophies",
                  "Chronic.Pain.Fibromyalgia",
                  "Alcohol.Abuse",
                  "Other.Substance.Abuse",
                  "Obesity",
                  "Schizophrenia.and.other.Psychiatric.Disorders",
                  "Depression"]

for condition in all_conditions:
    overall_results_dict_no_relu[condition] = {}
    labels = torch.tensor(labelled_corpus_df[condition]).to(device)

    dataset = CustomDatasetEmbedded(study_corpus_tensor, labels)

    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True) # Batch size?
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32)

    overall_results_dict_no_relu[condition]["[1]"] = train_test_model_no_ReLU(n_gram_sizes = [1],
                                                                              condition = condition,
                                                                              train_loader = train_loader,
                                                                              test_loader = test_loader,
                                                                              device = device,
                                                                              n_epochs = n_epochs)
    overall_results_dict_no_relu[condition]["[1, 2]"] = train_test_model_no_ReLU(n_gram_sizes = [1, 2],
                                                                                 condition = condition,
                                                                                 train_loader = train_loader,
                                                                                 test_loader = test_loader,
                                                                                 device = device,
                                                                                 n_epochs = n_epochs)
    overall_results_dict_no_relu[condition]["[1, 2, 3]"] = train_test_model_no_ReLU(n_gram_sizes = [1, 2, 3],
                                                                                    condition = condition,
                                                                                    train_loader = train_loader,
                                                                                    test_loader = test_loader,
                                                                                    device = device,
                                                                                    n_epochs = n_epochs)
    overall_results_dict_no_relu[condition]["[1, 2, 3, 4]"] = train_test_model_no_ReLU(n_gram_sizes = [1, 2, 3, 4],
                                                                                       condition = condition,
                                                                                       train_loader = train_loader,
                                                                                       test_loader = test_loader,
                                                                                       device = device,
                                                                                       n_epochs = n_epochs)
    overall_results_dict_no_relu[condition]["[1, 2, 3, 4, 5]"] = train_test_model_no_ReLU(n_gram_sizes = [1, 2, 3, 4, 5],
                                                                                          condition = condition,
                                                                                          train_loader = train_loader,
                                                                                          test_loader = test_loader,
                                                                                          device = device,
                                                                                          n_epochs = n_epochs)
    overall_results_dict_no_relu[condition]["[2, 3, 4, 5]"] = train_test_model_no_ReLU(n_gram_sizes = [2, 3, 4, 5],
                                                                                       condition = condition,
                                                                                       train_loader = train_loader,
                                                                                       test_loader = test_loader,
                                                                                       device = device,
                                                                                       n_epochs = n_epochs)

with open("no_ReLU_results.json", 'w') as json_file:
    json.dump(overall_results_dict_no_relu, json_file)

[1]-gram CNN for Advanced.Cancer
Training time: 14.53543758392334 seconds
Predicted percent of patients that have the condition: 0.018656716417910446
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8880597014925373
Precision: 0.8
Recall: 0.12121212121212122
F1 Score: 0.2105263157894737
AUC: 0.9128304319793682

[1, 2]-gram CNN for Advanced.Cancer
Training time: 20.361693859100342 seconds
Predicted percent of patients that have the condition: 0.10074626865671642
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.9402985074626866
Precision: 0.8148148148148148
Recall: 0.6666666666666666
F1 Score: 0.7333333333333333
AUC: 0.9519664732430689

[1, 2, 3]-gram CNN for Advanced.Cancer
Training time: 55.84630584716797 seconds
Predicted percent of patients that have the condition: 0.08582089552238806
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.9253731343283582
Precision: 0.782608695652174
Re

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 95.76671981811523 seconds
Predicted percent of patients that have the condition: 0.05223880597014925
Actual percent of patients that have the condition: 0.13805970149253732
Accuracy: 0.8843283582089553
Precision: 0.7142857142857143
Recall: 0.2702702702702703
F1 Score: 0.39215686274509803
AUC: 0.9197379197379199

[1, 2, 3, 4, 5]-gram CNN for Advanced.Heart.Disease
Training time: 123.6586971282959 seconds
Predicted percent of patients that have the condition: 0.06343283582089553
Actual percent of patients that have the condition: 0.13805970149253732
Accuracy: 0.8955223880597015
Precision: 0.7647058823529411
Recall: 0.35135135135135137
F1 Score: 0.48148148148148157
AUC: 0.9108459108459109

[2, 3, 4, 5]-gram CNN for Advanced.Heart.Disease
Training time: 117.65337562561035 seconds
Predicted percent of patients that have the condition: 0.05223880597014925
Actual percent of patients that have the condition: 0.13805970149253732
Accuracy: 0.8917910447761194
Precision: 0.785714285

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 21.276213884353638 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8768656716417911
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.8876853642811089

[1, 2, 3]-gram CNN for Advanced.Lung.Disease


...

Training time: 117.55360984802246 seconds
Predicted percent of patients that have the condition: 0.011194029850746268
Actual percent of patients that have the condition: 0.09328358208955224
Accuracy: 0.9104477611940298
Precision: 0.6666666666666666
Recall: 0.08
F1 Score: 0.14285714285714285
AUC: 0.7494650205761317

[1]-gram CNN for Schizophrenia.and.other.Psychiatric.Disorders
Training time: 8.549675703048706 seconds
Predicted percent of patients that have the condition: 0.018656716417910446
Actual percent of patients that have the condition: 0.19029850746268656
Accuracy: 0.8208955223880597
Precision: 0.8
Recall: 0.0784313725490196
F1 Score: 0.14285714285714285
AUC: 0.7812415288696124

[1, 2]-gram CNN for Schizophrenia.and.other.Psychiatric.Disorders
Training time: 21.276909589767456 seconds
Predicted percent of patients that have the condition: 0.08582089552238806
Actual percent of patients that have the condition: 0.19029850746268656
Accuracy: 0.8656716417910447
Precision: 0.82608695

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 123.68478798866272 seconds
Predicted percent of patients that have the condition: 0.0037313432835820895
Actual percent of patients that have the condition: 0.19029850746268656
Accuracy: 0.8134328358208955
Precision: 1.0
Recall: 0.0196078431372549
F1 Score: 0.038461538461538464
AUC: 0.7410318966296195

[2, 3, 4, 5]-gram CNN for Schizophrenia.and.other.Psychiatric.Disorders
Training time: 117.55957841873169 seconds
Predicted percent of patients that have the condition: 0.007462686567164179
Actual percent of patients that have the condition: 0.19029850746268656
Accuracy: 0.8171641791044776
Precision: 1.0
Recall: 0.0392156862745098
F1 Score: 0.07547169811320754
AUC: 0.7516942260775278

[1]-gram CNN for Depression
Training time: 8.502787590026855 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.27611940298507465
Accuracy: 0.7238805970149254
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.8706464196154918


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 21.296201467514038 seconds
Predicted percent of patients that have the condition: 0.2835820895522388
Actual percent of patients that have the condition: 0.27611940298507465
Accuracy: 0.8582089552238806
Precision: 0.7368421052631579
Recall: 0.7567567567567568
F1 Score: 0.7466666666666667
AUC: 0.9019573697408749

[1, 2, 3]-gram CNN for Depression
Training time: 54.98253083229065 seconds
Predicted percent of patients that have the condition: 0.06343283582089553
Actual percent of patients that have the condition: 0.27611940298507465
Accuracy: 0.7574626865671642
Precision: 0.7647058823529411
Recall: 0.17567567567567569
F1 Score: 0.28571428571428575
AUC: 0.8540679855112845

[1, 2, 3, 4]-gram CNN for Depression
Training time: 95.78844499588013 seconds
Predicted percent of patients that have the condition: 0.08955223880597014
Actual percent of patients that have the condition: 0.27611940298507465
Accuracy: 0.7686567164179104
Precision: 0.75
Recall: 0.24324324324324326
F1 Score: 

In [ ]:
class CNN_n_gram_no_pooling(nn.Module):
    def __init__(self, n_gram_sizes):
        super(CNN_n_gram_no_pooling, self).__init__()

        self.n_gram_sizes = n_gram_sizes
        num_sizes = 0

        total_convolutional_output_size = 0

        if 1 in self.n_gram_sizes:
            self.conv1 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (1, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)

            conv1_output_height = study_corpus_tensor.shape[1] + 1 - 1
            total_convolutional_output_size += conv1_output_height * kernels_per_n_gram_size
            num_sizes += 1

        if 2 in self.n_gram_sizes:
            self.conv2 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (2, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)

            conv2_output_height = study_corpus_tensor.shape[1] + 1 - 2
            total_convolutional_output_size += conv2_output_height * kernels_per_n_gram_size
            num_sizes += 1

        if 3 in self.n_gram_sizes:
            self.conv3 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (3, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)

            conv3_output_height = study_corpus_tensor.shape[1] + 1 - 3
            total_convolutional_output_size += conv3_output_height * kernels_per_n_gram_size
            num_sizes += 1

        if 4 in self.n_gram_sizes:
            self.conv4 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (4, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)

            conv4_output_height = study_corpus_tensor.shape[1] + 1 - 4
            total_convolutional_output_size += conv4_output_height * kernels_per_n_gram_size
            num_sizes += 1

        if 5 in self.n_gram_sizes:
            self.conv5 = nn.Conv2d(in_channels = 1,
                                  out_channels = kernels_per_n_gram_size,
                                  kernel_size = (5, embedding_vector_size),
                                  stride = 1,
                                  padding = 0)

            conv5_output_height = study_corpus_tensor.shape[1] + 1 - 5
            total_convolutional_output_size += conv5_output_height * kernels_per_n_gram_size
            num_sizes += 1

        assert num_sizes > 0 # I can't just assert that len(n_gram_sizes) > 0, as n_gram_sizes may contain unsupported values

        self.do = nn.Dropout(p = 0.5) 

        self.fc = nn.Linear(total_convolutional_output_size, 2) 

        self.activation = nn.LogSoftmax(dim = 1) # Called on tensor of shape [batch_size, 2]
        
    def forward(self, x):
        # Add an extra dimension, because nn.Conv2d takes an input of (batch_size, number of channels, height of input, width of input)
        # and not (batch_size, height of input, width of input)
        x = torch.unsqueeze(x, dim = 1)

        list_of_convolutional_outputs = []

        # For each n-gram size, apply the filters to the input, followed by ReLU
        # Adding the outputs of the convolutional layer and the following ReLU operation to a list
        # enables this class to be used for CNNs considering many different combinations of n-gram sizes
        if 1 in self.n_gram_sizes:
            x1 = self.conv1(x)
            x1 = torch.relu(x1) 
            x1 = torch.flatten(x1, start_dim = 1)
            list_of_convolutional_outputs.append(x1)
        
        if 2 in self.n_gram_sizes:
            x2 = self.conv2(x)
            x2 = torch.relu(x2)
            x2 = torch.flatten(x2, start_dim = 1)
            list_of_convolutional_outputs.append(x2)
        
        if 3 in self.n_gram_sizes:
            x3 = self.conv3(x)
            x3 = torch.relu(x3)
            x3 = torch.flatten(x3, start_dim = 1)
            list_of_convolutional_outputs.append(x3)

        if 4 in self.n_gram_sizes:
            x4 = self.conv4(x)
            x4 = torch.relu(x4)
            x4 = torch.flatten(x4, start_dim = 1)
            list_of_convolutional_outputs.append(x4)
        
        if 5 in self.n_gram_sizes:
            x5 = self.conv5(x)
            x5 = torch.relu(x5)
            x5 = torch.flatten(x5, start_dim = 1)
            list_of_convolutional_outputs.append(x5)

        # Combine the results of applying differently sized kernels in parallel
        x = torch.cat(list_of_convolutional_outputs, dim = 1)

        # Perform a dropout to reduce overfitting
        x = self.do(x)
        
        # Feed the (dropped out) outputs of the ReLU step into a fully connected layer,
        # and then a log-softmax activation function, to generate two values
        # If the first is greater, the patient is classified as not having the condition
        # otherwise, they are classified as having it. 
        x = self.fc(x)
        x = self.activation(x)

        return x

In [ ]:
# Analogous to train_test_model() earlier
def train_test_model_no_pooling(n_gram_sizes, condition, train_loader, test_loader, device, n_epochs):
    results_dict = {}

    which_model = str(n_gram_sizes) + "-gram CNN for " + condition
    print(which_model)

    model = CNN_n_gram_no_pooling(n_gram_sizes).to(device)

    criterion = nn.modules.loss.NLLLoss()

    optimizer = torch.optim.Adadelta(model.parameters(), rho = 0.95, eps = 1e-6)

    start_time = time.time()
    model = train_model(model = model,
                        train_dataloader = train_loader,
                        n_epoch = n_epochs,
                        optimizer = optimizer,
                        criterion = criterion,
                        normalize = False) # Because we have a lot more parameters than when we used global max pooling on the output of each filter, normalizing to 3 doesn't seem to make sense
    end_time = time.time()
    training_time = end_time - start_time
    training_time_statement = "Training time: " + str(training_time) + " seconds"
    print(training_time_statement)
    results_dict["training time"] = training_time

    y_score, y_pred, y_true = eval_model(model, test_loader)

    predicted_positive = np.sum(y_pred) / len(y_pred)
    predicted_positive_statement = "Predicted percent of patients that have the condition: " + str(predicted_positive)
    print(predicted_positive_statement)
    results_dict["predicted percent positive"] = predicted_positive

    actual_positive = np.sum(y_true) / len(y_true)
    actual_positive_statement = "Actual percent of patients that have the condition: " + str(actual_positive)
    print(actual_positive_statement)
    results_dict["actual percent positive"] = actual_positive

    accuracy = accuracy_score(y_true, y_pred)
    accuracy_statement = "Accuracy: " + str(accuracy)
    print(accuracy_statement)
    results_dict["accuracy"] = accuracy

    precision = precision_score(y_true, y_pred)
    precision_statement = "Precision: " + str(precision)
    print(precision_statement)
    results_dict["precision"] = precision

    recall = recall_score(y_true, y_pred)
    recall_statement = "Recall: " + str(recall)
    print(recall_statement)
    results_dict["recall"] = recall

    f1 = f1_score(y_true, y_pred)
    f1_statement = "F1 Score: " + str(f1)
    print(f1_statement)
    results_dict["f1"] = f1

    auc = roc_auc_score(y_true, y_score)
    auc_statement = "AUC: " + str(auc)
    print(auc_statement)
    results_dict["auc"] = auc

    print()
    return results_dict

In [ ]:
# Print the results of training and testing all 60 CNNs with pooling omitted, and save them to "no_pooling_results.json"

seed =  0
torch.manual_seed(seed)

overall_results_dict_no_pooling = {}

# advanced or metastatic cancer, advanced heart disease, advanced lung disease, chronic neurologic dystrophies, chronic pain, alcohol abuse, substance abuse, obesity, psychiatric disorders, or depression.
all_conditions = ["Advanced.Cancer",
                  "Advanced.Heart.Disease",
                  "Advanced.Lung.Disease",
                  "Chronic.Neurological.Dystrophies",
                  "Chronic.Pain.Fibromyalgia",
                  "Alcohol.Abuse",
                  "Other.Substance.Abuse",
                  "Obesity",
                  "Schizophrenia.and.other.Psychiatric.Disorders",
                  "Depression"]

for condition in all_conditions:
    overall_results_dict_no_pooling[condition] = {}
    labels = torch.tensor(labelled_corpus_df[condition]).to(device)

    dataset = CustomDatasetEmbedded(study_corpus_tensor, labels)

    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True) # Batch size?
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32)

    overall_results_dict_no_pooling[condition]["[1]"] = train_test_model_no_pooling(n_gram_sizes = [1],
                                                                                    condition = condition,
                                                                                    train_loader = train_loader,
                                                                                    test_loader = test_loader,
                                                                                    device = device,
                                                                                    n_epochs = n_epochs)
    overall_results_dict_no_pooling[condition]["[1, 2]"] = train_test_model_no_pooling(n_gram_sizes = [1, 2],
                                                                                       condition = condition,
                                                                                       train_loader = train_loader,
                                                                                       test_loader = test_loader,
                                                                                       device = device,
                                                                                       n_epochs = n_epochs)
    overall_results_dict_no_pooling[condition]["[1, 2, 3]"] = train_test_model_no_pooling(n_gram_sizes = [1, 2, 3],
                                                                                          condition = condition,
                                                                                          train_loader = train_loader,
                                                                                          test_loader = test_loader,
                                                                                          device = device,
                                                                                          n_epochs = n_epochs)
    overall_results_dict_no_pooling[condition]["[1, 2, 3, 4]"] = train_test_model_no_pooling(n_gram_sizes = [1, 2, 3, 4],
                                                                                             condition = condition,
                                                                                             train_loader = train_loader,
                                                                                             test_loader = test_loader,
                                                                                             device = device,
                                                                                             n_epochs = n_epochs)
    overall_results_dict_no_pooling[condition]["[1, 2, 3, 4, 5]"] = train_test_model_no_pooling(n_gram_sizes = [1, 2, 3, 4, 5],
                                                                                                condition = condition,
                                                                                                train_loader = train_loader,
                                                                                                test_loader = test_loader,
                                                                                                device = device,
                                                                                                n_epochs = n_epochs)
    overall_results_dict_no_pooling[condition]["[2, 3, 4, 5]"] = train_test_model_no_pooling(n_gram_sizes = [2, 3, 4, 5],
                                                                                             condition = condition,
                                                                                             train_loader = train_loader,
                                                                                             test_loader = test_loader,
                                                                                             device = device,
                                                                                             n_epochs = n_epochs)

with open("no_pooling_results.json", 'w') as json_file:
    json.dump(overall_results_dict_no_pooling, json_file)

[1]-gram CNN for Advanced.Cancer
Training time: 16.351748943328857 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8768656716417911
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.6370083816892327

[1, 2]-gram CNN for Advanced.Cancer


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 21.9148530960083 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8768656716417911
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.7400386847195358

[1, 2, 3]-gram CNN for Advanced.Cancer


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 57.352721214294434 seconds
Predicted percent of patients that have the condition: 0.0037313432835820895
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8731343283582089
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.5908446163765313

[1, 2, 3, 4]-gram CNN for Advanced.Cancer
Training time: 98.55386805534363 seconds
Predicted percent of patients that have the condition: 0.0037313432835820895
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8805970149253731
Precision: 1.0
Recall: 0.030303030303030304
F1 Score: 0.05882352941176471
AUC: 0.672340425531915

[1, 2, 3, 4, 5]-gram CNN for Advanced.Cancer
Training time: 129.24922847747803 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.12313432835820895
Accuracy: 0.8768656716417911
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.6567375886524822

[2, 3, 4, 5]-gram CNN for Advanced.Cancer


...

Training time: 22.851659774780273 seconds
Predicted percent of patients that have the condition: 0.0037313432835820895
Actual percent of patients that have the condition: 0.05970149253731343
Accuracy: 0.9365671641791045
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.5379464285714286

[1, 2, 3]-gram CNN for Obesity
Training time: 59.526813983917236 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.05970149253731343
Accuracy: 0.9402985074626866
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.4486607142857143

[1, 2, 3, 4]-gram CNN for Obesity


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 98.75567603111267 seconds
Predicted percent of patients that have the condition: 0.0037313432835820895
Actual percent of patients that have the condition: 0.05970149253731343
Accuracy: 0.9365671641791045
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.4685019841269842

[1, 2, 3, 4, 5]-gram CNN for Obesity
Training time: 129.16360783576965 seconds
Predicted percent of patients that have the condition: 0.0
Actual percent of patients that have the condition: 0.05970149253731343
Accuracy: 0.9402985074626866
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.5178571428571428

[2, 3, 4, 5]-gram CNN for Obesity


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training time: 121.07337474822998 seconds
Predicted percent of patients that have the condition: 0.007462686567164179
Actual percent of patients that have the condition: 0.05970149253731343
Accuracy: 0.9328358208955224
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.4397321428571429

[1]-gram CNN for Schizophrenia.and.other.Psychiatric.Disorders
Training time: 9.305625200271606 seconds
Predicted percent of patients that have the condition: 0.011194029850746268
Actual percent of patients that have the condition: 0.16791044776119404
Accuracy: 0.8208955223880597
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
AUC: 0.5327354260089686

[1, 2]-gram CNN for Schizophrenia.and.other.Psychiatric.Disorders
Training time: 22.870659589767456 seconds
Predicted percent of patients that have the condition: 0.026119402985074626
Actual percent of patients that have the condition: 0.16791044776119404
Accuracy: 0.8283582089552238
Precision: 0.42857142857142855
Recall: 0.06666666666666667
F1 Score: 0.115384615384